In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [27]:
driver = webdriver.Chrome()
driver.get('https://comic.naver.com/webtoon?tab=mon')

In [59]:
# 얻어야 할 것: title, author, genre, describtion, rating
html = driver.page_source # 가져온 페이지의 source를 저장
soup = BeautifulSoup(html, 'html.parser')
items = soup.select("#content > div:nth-child(1) > ul > li")
base_link = "https://comic.naver.com/"
for item in items:
    detail_link = base_link + item.select_one("a.ContentTitle__title_area--x24vt")['href']
    

https://comic.naver.com//webtoon/list?titleId=648419&tab=mon
https://comic.naver.com//webtoon/list?titleId=758037&tab=mon
https://comic.naver.com//webtoon/list?titleId=733074&tab=mon
https://comic.naver.com//webtoon/list?titleId=183559&tab=mon
https://comic.naver.com//webtoon/list?titleId=817859&tab=mon
https://comic.naver.com//webtoon/list?titleId=602910&tab=mon
https://comic.naver.com//webtoon/list?titleId=783052&tab=mon
https://comic.naver.com//webtoon/list?titleId=654774&tab=mon
https://comic.naver.com//webtoon/list?titleId=795297&tab=mon
https://comic.naver.com//webtoon/list?titleId=796867&tab=mon
https://comic.naver.com//webtoon/list?titleId=796075&tab=mon
https://comic.naver.com//webtoon/list?titleId=774863&tab=mon
https://comic.naver.com//webtoon/list?titleId=822573&tab=mon
https://comic.naver.com//webtoon/list?titleId=806189&tab=mon
https://comic.naver.com//webtoon/list?titleId=822657&tab=mon
https://comic.naver.com//webtoon/list?titleId=813552&tab=mon
https://comic.naver.com/

In [ ]:
for item in items:
    # 제목 추출
    title = item.select_one("div > a > span > span").text.strip()
    
    # 저자 정보가 있는 여러 위치를 체크하여 첫 번째로 발견된 요소를 사용
    author_element = item.select_one("div > div.ContentAuthor__author_wrap--fV7Lo > a, div > a.ContentAuthor__author--CTAAP")
    author = author_element.text.strip() if author_element else "Author Not Found"
    
    # 평점 추출
    rating_element = item.select_one("div > div.rating_area > span > span")
    rating = rating_element.text.strip() if rating_element else "Rating Not Found"
    
    print(title, author, rating)

In [25]:
driver.close()

In [51]:
class scraper:
    def __init__(self, url):
        self.url = url
        self.driver = webdriver.Chrome()
        self.soup = None
        self.data = []

    def fetchPage(self):
        self.driver.get(self.url)
        time.sleep(3)
        self.soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        
    def parseTAR(self): # T: title, A: author, R: rating
        if self.soup is not None:
            items = self.soup.select("#content > div:nth-child(1) > ul > li")
            for item in items:
                # 제목 추출
                title = item.select_one("div > a > span > span").text.strip()
                
                # 저자 정보가 있는 여러 위치를 체크하여 첫 번째로 발견된 요소를 사용
                author_element = item.select_one("div > div.ContentAuthor__author_wrap--fV7Lo > a, div > a.ContentAuthor__author--CTAAP")
                author = author_element.text.strip() if author_element else "Author Not Found"
                
                # 평점 추출
                rating = item.select_one("div > div.rating_area > span > span")

                self.data.append({
                'title': title,
                'author': author,
                'rating': rating})
        else:
            print("Soup is None. Call fetch_page() first")
    def parseGD(self): # G: genre, D: description
        if self.soup is not None:
            items 
                            

In [ ]:
#content > div:nth-child(1) > ul > li:nth-child(1) > div > a > span > span
#content > div:nth-child(1) > ul > li:nth-child(2) > div > a > span > span

#content > div:nth-child(1) > ul > li:nth-child(1) > div > div.ContentAuthor__author_wrap--fV7Lo > a
#content > div:nth-child(1) > ul > li:nth-child(2) > div > div.ContentAuthor__author_wrap--fV7Lo > a

#content > div:nth-child(1) > ul > li:nth-child(1) > div > div.rating_area > span > span
#content > div:nth-child(1) > ul > li:nth-child(2) > div > div.rating_area > span > span

#content > div:nth-child(1) > ul > li:nth-child(3) > div > a.ContentAuthor__author--CTAAP

#content > div:nth-child(1) > ul > li:nth-child(1) > div > a > span